# RPC & Pointers

## Launching the domain

In [ ]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --dev --tail

In [2]:
import hagrid
hagrid.check("localhost:8081")

Output()

✅ test_domain Domain Containers Created

✅ Backend

✅ Startup Complete

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ PyGrid    ┃ Info                                      ┃ 600 ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ❌  │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅  │
└───────────┴───────────────────────────────────────────┴─────┘

In [4]:
import syft as sy
sy.requires("==0.7")
domain_client = sy.login(
    port=8081,
    email="info@openmined.org",
    password="changethis"
)

✅ The installed version of syft==0.7.0 matches the requirement ==0.7

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test_domain... done!


In [44]:
domain_client.address

<Address - Domain:<SpecificLocation:..b4b6f>>

Initial store

In [ ]:
domain_client.store

## Making 2 pointers to 2 integers

In [ ]:
# ⚔️ Runnable Code
one = sy.lib.python.Int(1)
one_ptr = one.send(domain_client)
one_ptr

In [ ]:
two_ptr = domain_client.syft.lib.python.Int(2)
two_ptr

In [12]:
domain_client.store

,ID,Tags,Description,object_type
0,<UID: ec900f7c95c1444186e343010922b2d0>,[],,<class 'syft.lib.python.Int'>
1,<UID: da2118bb04024ccf8fa4f1530130d3d3>,[],,<class 'syft.lib.python.Int'>
2,<UID: b695c3e0a99d478bb744e345ed1cd9e8>,[],,<class 'syft.lib.python.Int'>


In [23]:
one_ptr = domain_client.store['da2118bb04024ccf8fa4f1530130d3d3']
two_ptr = domain_client.store['b695c3e0a99d478bb744e345ed1cd9e8']

In [24]:
one_ptr

<IntPointer -> test_domain:da2118bb04024ccf8fa4f1530130d3d3, status=Ready>

In [25]:
two_ptr

<IntPointer -> test_domain:b695c3e0a99d478bb744e345ed1cd9e8, status=Ready>

In [26]:
one_ptr.get(delete_obj=False)

1

In [27]:
two_ptr.get(delete_obj=False)

2

## AST

In [ ]:
domain_client.lib_ast

In [16]:
type(one_ptr)

syft.proxy.syft.lib.python.IntPointer

In [17]:
dir(one_ptr)

['__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__itruediv__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__name__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__weakref__',
 '__x

In [28]:
a = one_ptr
b = two_ptr
c = a + b

In [29]:
c

<IntPointer -> test_domain:85c420b0c65e4a4c90a2e71e837dd53d, status=Ready>

In [31]:
domain_client.store

,ID,Tags,Description,object_type
0,<UID: ec900f7c95c1444186e343010922b2d0>,[],,<class 'syft.lib.python.Int'>
1,<UID: 85c420b0c65e4a4c90a2e71e837dd53d>,[],,<class 'syft.lib.python.Int'>
2,<UID: da2118bb04024ccf8fa4f1530130d3d3>,[],,<class 'syft.lib.python.Int'>
3,<UID: b695c3e0a99d478bb744e345ed1cd9e8>,[],,<class 'syft.lib.python.Int'>


In [32]:
c.get(delete_obj=False)

3

In [35]:
d = c * b

In [38]:
d.get(delete_obj=False)

6

In [39]:
e = d / 2
e

<FloatPointer -> test_domain:4a96ebf186e343f0b3c1c6fb442eb0dc, status=Processing>

In [40]:
e.get(delete_obj=False)

3.0

## Under the hood of pointers division

Let's manually construct an action message to divide the pointer `d` by 2

Create a `UID` for the key to store our result under

In [41]:
uid = sy.common.UID()
uid

<UID: 10d92d1a8d6548e8ad84c73e713cca2f>

Let's construct the `action` we want which requires
- `path`: The location of the real `function` which we bind to a set of module paths nested inside the `syft` module
- `_self`: what would be bound to `self` if the function is a method
- `args` and `kwars`: Allows us to pass in anything that would normally be passed into the function as `*args` or `*kwargs`
- `id_at_location`: The result of any computation will be boxed back up as a `serializable` object and set `value` of the key supplied as `id_at_location`
- `address`: The message has the `address` of the `domain` we are messaging

In [45]:
from syft.core.node.common.action.run_class_method_action import RunClassMethodAction

cmd = RunClassMethodAction(
    path="syft.lib.python.Int.__truediv__",  # our desired function
    _self=d,  # the self for the method
    args=[two_ptr],  # the first arg for the method
    kwargs={},
    id_at_location=uid,  # the uid to store the result under
    address=domain_client.address,  # the node to process the work on
)
cmd

RunClassMethodAction IntPointer.__truediv__(IntPointer, )

Send the `message` to the `domain`

In [46]:
domain_client.send_eventual_msg_without_reply(msg=cmd)

The `message` is not responsible for `pointer` creation. Pointers are usually synthesized by the code which creates the `action` so we will need to make our own by hand

In [47]:
e_ptr = domain_client.lib_ast.query("syft.lib.python.Float").pointer_type(
    client=domain_client,
    id_at_location=uid,
)
e_ptr

<FloatPointer -> test_domain:10d92d1a8d6548e8ad84c73e713cca2f, status=Ready>

We can see that the `pointers` are bound to a particular `client`

In [48]:
e_ptr.get(delete_obj=False)

3.0

In `syft`, `execution`'s result by a client on non-readable `data` is always stored remotely out of reach of the `client` => `data` derived from `private` data will automatically be `private` and will not be `read`able by the client who created it

In [49]:
guest_client = sy.login(
    port=8081,
)

Connecting to localhost... done! 	 Logging into test_domain... as GUEST...done!


Use `UID` to get a new pointer to the same object but with different client permissions

In [54]:
a_uid = a.id_at_location.no_dash
a_uid

'da2118bb04024ccf8fa4f1530130d3d3'

In [55]:
guest_a = guest_client.store[a_uid]
guest_a

<IntPointer -> test_domain:da2118bb04024ccf8fa4f1530130d3d3, status=Ready>

Try some addition

In [56]:
x_ptr = guest_a + guest_a

That worked. But can we get the result?

In [58]:
try:
    x_ptr.get(delete_obj=True)
except Exception:
    print("guest client cannot get the result")
    pass

guest client cannot get the result
<class 'Exception'>


## Datasets

In [62]:
import numpy as np
from syft.core.adp.data_subject import DataSubject

example_data = np.array([1, 2, 3])
example_dataset = sy.Tensor(example_data)
example_dataset.public_shape = example_data.shape
example_dataset = example_dataset.annotate_with_dp_metadata(
    lower_bound=0,
    upper_bound=3,
    data_subject=DataSubject("Alice")
).tag("example_data")

Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.


In [63]:
domain_client.load_dataset(
    assets={"example_dataset": example_dataset},
    name="An example dataset",
    description=(
        "A collection of numbers about people"
    )
)

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/dk


Loading dataset... uploading...🚀                                                                                                                                             

Uploading `example_dataset`: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  4.52it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [64]:
domain_client.datasets

Idx,Name,Description,Assets,Id
[0],An example dataset,A collection of numbers about people,"[""example_dataset""] -> Tensor",ac0062f9-1770-476c-bf3c-37f87c7ef515
